In [ ]:
pip install pytorch-widedeep

In [1]:
import os
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import matplotlib.font_manager as fm
import matplotlib
import itertools


from itertools import product  
from math import sqrt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep
from pytorch_widedeep.metrics import Accuracy
from pytorch_widedeep import Trainer

In [2]:
FILES_DIR = '../data/a1/'
log_df = pd.read_csv(FILES_DIR + 'log_csv.csv')

In [20]:
# null값들을 un_** 으로 채워서 확인해보기
log_drop_null = log_df
log_drop_null['geoip_city_name'].fillna('un_city_name', inplace = True)
log_drop_null['category3'].fillna('un_category', inplace = True)
log_drop_null['uid'].fillna('un_name', inplace = True)
log_drop_null.dropna(axis = 0, inplace = True)
print(log_drop_null.isnull().sum().sort_values(ascending=False)) # null값 퍼센트로 표현

Unnamed: 0         0
uid                0
action_type        0
category1          0
category2          0
category3          0
collect_time       0
geoip_city_name    0
useragent_os       0
viewrate           0
viewtime           0
rating             0
dtype: int64


In [21]:
log_drop_null['rating'] = log_drop_null['action_type']
re_name = {'rating' : {'View':1, 'Highlight':2, 'Basket':4, 'Copy':3, 'SINF':0}}
log_drop_null = log_drop_null.replace(re_name)
log_drop_null

,Unnamed: 0,uid,action_type,category1,category2,category3,collect_time,geoip_city_name,useragent_os,viewrate,viewtime,rating
0,0,avpKyWMn/xEADNlb,View,주방용품,그릇·홈세트,식판·나눔접시,2022-12-29 17:29:02,Seongnam-si,Windows,100.0,80033,1
1,1,0+5vEmGkOuAAB5j2,Basket,패브릭,매트,주방·다용도매트,2022-12-29 12:56:38,Yongin-si,Windows,0.0,0,4
2,2,r8DXy2OtSl8AAojG,View,패브릭,커튼·부자재,암막커튼,2022-12-29 17:11:30,Seoul,iOS,100.0,947,1
3,3,dC9Fl2Ly9lcADrTn,Highlight,데코·식물,플라워·식물,화병·화분,2022-12-29 17:54:30,Gwangmyeong-si,Windows,0.0,0,2
4,4,09gKcGLg0I8ACyQy,View,가구,진열장·책장,책장,2022-12-29 18:45:05,Jongno-gu,Mac OS X,100.0,1238,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5659536,5659536,PUpxWWMWdcIACR2B,View,주방용품,칼·커팅기구,채칼·다지기·강판,2022-12-28 20:01:30,Seoul,Android,100.0,13893,1
5659537,5659537,PVMstmNg4I4ADq9R,Basket,가구,진열장·책장,책장,2022-12-28 14:30:09,Jinju,Windows,0.0,0,4
5659538,5659538,diRXdmN49dUABrVo,View,가구,테이블·식탁·책상,식탁,2022-12-29 12:45:13,Gunpo,Android,100.0,47217,1
5659539,5659539,dusFzWJvp5cADhZW,View,가전·디지털,계절가전,전기히터·온풍기,2022-12-28 11:07:30,Seoul,Android,100.0,54030,1


In [22]:
# 날짜 데이터 데이트타임으로 바꾸기
def _trans_date_time(log):
    log.index = pd.to_datetime(log['collect_time'], infer_datetime_format=True)
    log.drop('collect_time', axis=1, inplace=True)
    return log

_trans_date_time(log_drop_null)

C:\Users\bongkyun\AppData\Local\Temp\ipykernel_7804\1120596358.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  log.index = pd.to_datetime(log['collect_time'], infer_datetime_format=True)


,Unnamed: 0,uid,action_type,category1,category2,category3,geoip_city_name,useragent_os,viewrate,viewtime,rating
collect_time,,,,,,,,,,,
2022-12-29 17:29:02,0,avpKyWMn/xEADNlb,View,주방용품,그릇·홈세트,식판·나눔접시,Seongnam-si,Windows,100.0,80033,1
2022-12-29 12:56:38,1,0+5vEmGkOuAAB5j2,Basket,패브릭,매트,주방·다용도매트,Yongin-si,Windows,0.0,0,4
2022-12-29 17:11:30,2,r8DXy2OtSl8AAojG,View,패브릭,커튼·부자재,암막커튼,Seoul,iOS,100.0,947,1
2022-12-29 17:54:30,3,dC9Fl2Ly9lcADrTn,Highlight,데코·식물,플라워·식물,화병·화분,Gwangmyeong-si,Windows,0.0,0,2
2022-12-29 18:45:05,4,09gKcGLg0I8ACyQy,View,가구,진열장·책장,책장,Jongno-gu,Mac OS X,100.0,1238,1
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 20:01:30,5659536,PUpxWWMWdcIACR2B,View,주방용품,칼·커팅기구,채칼·다지기·강판,Seoul,Android,100.0,13893,1
2022-12-28 14:30:09,5659537,PVMstmNg4I4ADq9R,Basket,가구,진열장·책장,책장,Jinju,Windows,0.0,0,4
2022-12-29 12:45:13,5659538,diRXdmN49dUABrVo,View,가구,테이블·식탁·책상,식탁,Gunpo,Android,100.0,47217,1


In [23]:
log_drop_null["Hour"] = log_drop_null.index.hour

In [24]:
log_drop_null

,Unnamed: 0,uid,action_type,category1,category2,category3,geoip_city_name,useragent_os,viewrate,viewtime,rating,Hour
collect_time,,,,,,,,,,,,
2022-12-29 17:29:02,0,avpKyWMn/xEADNlb,View,주방용품,그릇·홈세트,식판·나눔접시,Seongnam-si,Windows,100.0,80033,1,17
2022-12-29 12:56:38,1,0+5vEmGkOuAAB5j2,Basket,패브릭,매트,주방·다용도매트,Yongin-si,Windows,0.0,0,4,12
2022-12-29 17:11:30,2,r8DXy2OtSl8AAojG,View,패브릭,커튼·부자재,암막커튼,Seoul,iOS,100.0,947,1,17
2022-12-29 17:54:30,3,dC9Fl2Ly9lcADrTn,Highlight,데코·식물,플라워·식물,화병·화분,Gwangmyeong-si,Windows,0.0,0,2,17
2022-12-29 18:45:05,4,09gKcGLg0I8ACyQy,View,가구,진열장·책장,책장,Jongno-gu,Mac OS X,100.0,1238,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 20:01:30,5659536,PUpxWWMWdcIACR2B,View,주방용품,칼·커팅기구,채칼·다지기·강판,Seoul,Android,100.0,13893,1,20
2022-12-28 14:30:09,5659537,PVMstmNg4I4ADq9R,Basket,가구,진열장·책장,책장,Jinju,Windows,0.0,0,4,14
2022-12-29 12:45:13,5659538,diRXdmN49dUABrVo,View,가구,테이블·식탁·책상,식탁,Gunpo,Android,100.0,47217,1,12


In [25]:
log = log_drop_null.drop_duplicates()
log

,Unnamed: 0,uid,action_type,category1,category2,category3,geoip_city_name,useragent_os,viewrate,viewtime,rating,Hour
collect_time,,,,,,,,,,,,
2022-12-29 17:29:02,0,avpKyWMn/xEADNlb,View,주방용품,그릇·홈세트,식판·나눔접시,Seongnam-si,Windows,100.0,80033,1,17
2022-12-29 12:56:38,1,0+5vEmGkOuAAB5j2,Basket,패브릭,매트,주방·다용도매트,Yongin-si,Windows,0.0,0,4,12
2022-12-29 17:11:30,2,r8DXy2OtSl8AAojG,View,패브릭,커튼·부자재,암막커튼,Seoul,iOS,100.0,947,1,17
2022-12-29 17:54:30,3,dC9Fl2Ly9lcADrTn,Highlight,데코·식물,플라워·식물,화병·화분,Gwangmyeong-si,Windows,0.0,0,2,17
2022-12-29 18:45:05,4,09gKcGLg0I8ACyQy,View,가구,진열장·책장,책장,Jongno-gu,Mac OS X,100.0,1238,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 20:01:30,5659536,PUpxWWMWdcIACR2B,View,주방용품,칼·커팅기구,채칼·다지기·강판,Seoul,Android,100.0,13893,1,20
2022-12-28 14:30:09,5659537,PVMstmNg4I4ADq9R,Basket,가구,진열장·책장,책장,Jinju,Windows,0.0,0,4,14
2022-12-29 12:45:13,5659538,diRXdmN49dUABrVo,View,가구,테이블·식탁·책상,식탁,Gunpo,Android,100.0,47217,1,12


In [26]:
log_sample = log.sample(frac=0.01, random_state=42)
log_sample = log_sample[~log_sample.index.duplicated(keep='first')]
log_sample

,Unnamed: 0,uid,action_type,category1,category2,category3,geoip_city_name,useragent_os,viewrate,viewtime,rating,Hour
collect_time,,,,,,,,,,,,
2022-12-27 14:39:16,3595088,bgojimOqhQEACbYY,View,가전·디지털,휴대폰,휴대폰,Seoul,Android,100.0,3006,1,14
2022-12-26 10:16:23,966238,0zPC5WOId8YABX08,Copy,가전·디지털,계절가전,전기요·온수매트,Yeongdeungpo-gu,Mac OS X,0.0,0,3,10
2022-12-26 13:08:35,2237030,t2umo2IVzewAAJiY,Highlight,패브릭,홈패브릭,홈웨어,Daejeon,Windows,0.0,0,2,13
2022-12-24 03:11:27,868149,AeM3zWOgVs4ABfZU,View,패브릭,러그·카페트,극세사·단모러그,Yongin-si,iOS,100.0,3906,1,3
2022-12-30 13:39:20,4828400,cJLHnmOuWO4AA50D,View,노하우,생활정보,un_category,Seongdong-gu,Mac OS X,100.0,44363,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 17:05:46,1821592,c1+lJGN3POgADwNi,View,가구,거울,전신거울,un_city_name,Windows,100.0,46759,1,17
2022-12-30 12:33:24,2919103,eaIjAmOuWoEACNHp,Highlight,가구,침대,침대프레임,Mapo-gu,Windows,0.0,0,2,12
2022-12-28 19:13:40,2997600,dtqrEmJUEyMADjG0,View,가구,선반,스탠드선반,Gunsan,Windows,0.0,0,1,19


In [28]:
log_uid = log_sample[['uid', 'action_type', 'viewrate', 'viewtime', 'rating', 'Hour']]
log_uid

,uid,action_type,viewrate,viewtime,rating,Hour
collect_time,,,,,,
2022-12-27 14:39:16,bgojimOqhQEACbYY,View,100.0,3006,1,14
2022-12-26 10:16:23,0zPC5WOId8YABX08,Copy,0.0,0,3,10
2022-12-26 13:08:35,t2umo2IVzewAAJiY,Highlight,0.0,0,2,13
2022-12-24 03:11:27,AeM3zWOgVs4ABfZU,View,100.0,3906,1,3
2022-12-30 13:39:20,cJLHnmOuWO4AA50D,View,100.0,44363,1,13
...,...,...,...,...,...,...
2022-12-28 17:05:46,c1+lJGN3POgADwNi,View,100.0,46759,1,17
2022-12-30 12:33:24,eaIjAmOuWoEACNHp,Highlight,0.0,0,2,12
2022-12-28 19:13:40,dtqrEmJUEyMADjG0,View,0.0,0,1,19


In [29]:
train_df, test_df = train_test_split(log_uid, test_size=0.2, random_state=42)

print(train_df.shape)
print(test_df.shape)

(42901, 6)
(10726, 6)


In [34]:
train_df

,uid,action_type,viewrate,viewtime,rating,Hour
collect_time,,,,,,
2022-12-28 15:14:35,fG9bBGLLvU8AAhSw,View,100.0,33229,1,15
2022-12-28 15:47:58,2pHmumMiuk8AA15U,View,100.0,6558,1,15
2022-12-27 15:43:05,cJv7YmOqdsYABmgZ,View,100.0,7113,1,15
2022-12-29 18:01:23,2pxI+GA26uIABMoQ,View,100.0,48488,1,18
2022-12-27 15:39:55,2ia/BWGAy88ACEXT,Highlight,0.0,0,2,15
...,...,...,...,...,...,...
2022-12-29 10:48:37,PUrdQGLwkoMADK5q,View,100.0,38359,1,10
2022-12-27 16:21:15,ebVcK2BRsa8ABaR8,View,100.0,1030,1,16
2022-12-26 16:34:48,diHkOWJwkoYABHbU,View,100.0,7015,1,16


In [30]:
category1 = [(list(set(x['uid'].values))[0], '/'.join(x['category1'].values)) for index, x in log_sample.groupby('uid')]
category1_df = pd.DataFrame(data=category1, columns=['uid', 'category1'])
category1_df = category1_df.set_index('uid')


category2 = [(list(set(x['uid'].values))[0], '/'.join(x['category2'].values)) for index, x in log_sample.groupby('uid')]
category2_df = pd.DataFrame(data=category2, columns=['uid', 'category2'])
category2_df = category2_df.set_index('uid')


countries = [(list(set(x['uid'].values))[0], ','.join(x['geoip_city_name'].values)) for index, x in log_sample.groupby('uid')]
countries_df = pd.DataFrame(data=countries, columns=['uid', 'country'])
countries_df = countries_df.set_index('uid')


os = [(list(set(x['uid'].values))[0], ','.join(x['useragent_os'].values)) for index, x in log_sample.groupby('uid')]
os_df = pd.DataFrame(data=os, columns=['uid', 'OS'])
os_df = os_df.set_index('uid')



log_total = pd.concat([category1_df, category2_df, countries_df, os_df], axis=1)

print(log_total.shape)
print(log_total.head())

(38984, 4)
                 category1 category2       country       OS
uid                                                        
0+0WhGBqwLQADiDj        가구     행거·옷장      Gimpo-si  Windows
0+0bBWJwbVkACg+Z        조명  무드등·장식조명      Gimpo-si  Windows
0+0hA2OFU6AADGdD    가전·디지털   생활·건강가전  un_city_name  Windows
0+157mLfUpoABHkY        가구   거실장·TV장     Goyang-si  Windows
0+2w2mGmCYIADF71     데코·식물    플라워·식물        Jeonju  Android


In [31]:
dummy_cat1_df = log_total['category1'].str.get_dummies(sep='/')
dummy_cat1_df

dummy_cat2_df = log_total['category2'].str.get_dummies(sep='/')
dummy_cat2_df

dummy_country_df = log_total['country'].str.get_dummies(sep=',')
dummy_country_df

dummy_os_df = log_total['OS'].str.get_dummies(sep=',')
dummy_os_df

,Android,CentOS,Chrome OS,Linux,Mac OS X,Other,Tizen,Ubuntu,Windows,iOS
uid,,,,,,,,,,
0+0WhGBqwLQADiDj,0,0,0,0,0,0,0,0,1,0
0+0bBWJwbVkACg+Z,0,0,0,0,0,0,0,0,1,0
0+0hA2OFU6AADGdD,0,0,0,0,0,0,0,0,1,0
0+157mLfUpoABHkY,0,0,0,0,0,0,0,0,1,0
0+2w2mGmCYIADF71,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
z/RHUGONyzUAAJRY,0,0,0,0,1,0,0,0,0,0
zbneuGOsO/IABD68,1,0,0,0,0,0,0,0,0,0
znTIGWMvY6UAC6BG,0,0,0,0,0,0,0,0,1,0


In [37]:
X_train = pd.concat([pd.get_dummies(train_df['uid'], prefix='uid'),
           train_df['uid'].apply(lambda x: dummy_cat1_df.loc[x]),
           train_df['uid'].apply(lambda x: dummy_cat2_df.loc[x]),
           train_df['uid'].apply(lambda x: dummy_country_df.loc[x]),
           train_df['uid'].apply(lambda x: dummy_os_df.loc[x])], axis=1)